# 0.0. IMPORTS

## 0.1. Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os

from unidecode import unidecode

C:\Users\Lucas\AppData\Local\Temp\ipykernel_9936\670289907.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 0.2. Carregar Dados

### 0.2.1. Escolas

In [2]:
diretorio_escolas = os.path.dirname(os.path.abspath('escolas.csv'))
caminho_escolas = os.path.join(diretorio_escolas, '../..', 'data', 'raw', 'escolas.csv')
df_escolas_raw = pd.read_csv(caminho_escolas)

### 0.2.2. Material Didatico

In [3]:
diretorio_material_didatico = os.path.dirname(os.path.abspath('material_didatico.csv'))
caminho_material_didatico = os.path.join(diretorio_material_didatico, '../..', 'data', 'raw', 'material_didatico.csv')
df_material_didatico_raw = pd.read_csv(caminho_material_didatico)

### 0.2.3. Sub-Prefeituras

In [4]:
diretorio_subprefeituras = os.path.dirname(os.path.abspath('subprefeituras.csv'))
caminho_subprefeituras = os.path.join(diretorio_subprefeituras, '../..', 'data', 'raw', 'subprefeituras.csv')
df_subprefeituras_raw = pd.read_csv(caminho_subprefeituras)

## 0.3. Funções

### 0.3.1. renomeia_colunas

In [5]:
def renomeia_colunas(data):
    return [x.lower() for x in data.columns]

### 0.3.2. tamanho_dos_dados

In [6]:
def tamanho_dos_dados(data):
    print(f'Colunas: {data.shape[1]}')
    print(f'Linhas: {data.shape[0]}')

### 0.3.3. qtd_total_unicos_por_coluna

In [7]:
def qtd_total_unicos_por_coluna(data):
    for x in data.columns:
        print(f'Quantidade de valores unicos em {x}: {len(data[x].unique())}')

### 0.3.4. get_numericos

In [8]:
def get_numericos(data):
    return data.select_dtypes(include=['int32', 'int64', 'float32', 'float64'])

### 0.3.5. get_categoricos

In [9]:
def get_categoricos(data):
    return data.select_dtypes(exclude=['int32', 'int64', 'float32', 'float64', 'datetime64[ns]'])

### 0.3.6. get_descriptive_info

In [10]:
def get_descriptive_info(data):
    dados_numericos = get_numericos(data)
    
    # medidas de tendencia central - media, mediana
    df_media = pd.DataFrame(dados_numericos.apply(np.mean)).T
    df_mediana = pd.DataFrame(dados_numericos.apply(np.median)).T   
    
    # medidas de dispersão - min, max, range, skew, kurtosis, std
    df_std = pd.DataFrame(dados_numericos.apply(np.std)).T
    df_min = pd.DataFrame(dados_numericos.apply(min)).T
    df_max = pd.DataFrame(dados_numericos.apply(max)).T
    df_range = pd.DataFrame(dados_numericos.apply(lambda x: x.max() - x.min())).T
    df_skew = pd.DataFrame(dados_numericos.apply(lambda x: x.skew())).T
    df_kurtosis = pd.DataFrame(dados_numericos.apply(lambda x: x.kurtosis())).T
    
    #min, max, range, media, mediana, std, skew, kurtosis
    df_descriptive_info = pd.concat([df_min, df_max, df_range,
                                     df_media, df_mediana, df_std,
                                     df_skew, df_kurtosis]).T.reset_index()
    
    df_descriptive_info.columns = ['attributes', 'min', 'max',
                                   'range', 'media', 'mediana',
                                   'std', 'skew', 'kurtosis']
    
    return df_descriptive_info

### 0.3.7. salvar_dados_csv

In [11]:
def salvar_dados_csv(data, nome=''):
    data.to_csv(f'../../data/processed/{nome}', index=False )

### 0.3.8. remove_acentos

In [12]:
def remove_acentos(data):
    categoricos = get_categoricos(data)
    for column in categoricos.columns:        
        data[column] = data[column].apply(lambda x: unidecode(x))
    return data

### 0.3.9. string_maisculo

In [13]:
def string_maisculo(data):
    categoricos = get_categoricos(data)

    for column in categoricos:
        data[column] = data[column].str.upper()
        
    return data

### 0.3.10. remodelando_id

In [14]:
def remodelando_id(data):
    data['id'] = data['id'].astype(str).apply(lambda x: f'0{x}' if len(x) < 3 else f'{x}')
    return data

# 1.0. DESCRIÇÃO DOS DADOS

## 1.1. Escolas

### 1.1.1. Visão Geral dos Dados

In [15]:
df1_escolas = df_escolas_raw.copy()

In [16]:
df1_escolas.head().T

,0,1,2,3,4
id,178,634,483,476,132
Escolas_Postos,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,EM ALICE DO AMARAL PEIXOTO,EM CELESTINO SILVA,ESCOLA MUNICIPAL FLORIANO PEIXOTO,EM PEREIRA PASSOS
BAIRRO,CAJU,BENFICA,CENTRO,SÃO CRISTÓVÃO,RIO COMPRIDO
ENDEREÇO,Rua Carlos Seidl S/Nº,Rua Ébano 187,"R. do Lavradio, 56",Praça Argentina 20,Praça Condessa Paulo de Frontin 45
lat,"-22,880888","-22,889574","-22,909293","-22,897629","-22,924412"
lon,"-43,225326","-43,236202","-43,183579","-43,227456","-43,208579"


In [17]:
df1_escolas.columns = renomeia_colunas(df1_escolas)

In [18]:
df1_escolas.columns

Index(['id', 'escolas_postos', 'bairro', 'endereço ', 'lat', 'lon'], dtype='object')

### 1.1.2. Tamanho dos dados

In [19]:
tamanho_dos_dados(df1_escolas)

Colunas: 6
Linhas: 152


In [20]:
qtd_total_unicos_por_coluna(df1_escolas)

Quantidade de valores unicos em id: 136
Quantidade de valores unicos em escolas_postos: 136
Quantidade de valores unicos em bairro: 90
Quantidade de valores unicos em endereço : 137
Quantidade de valores unicos em lat: 136
Quantidade de valores unicos em lon: 135


### 1.1.3. Verificar Nulos

In [21]:
df1_escolas.isna().sum()

id                0
escolas_postos    0
bairro            0
endereço          0
lat               0
lon               0
dtype: int64

### 1.1.4. Verificar tipos dos dados

In [22]:
df1_escolas.dtypes

id                 int64
escolas_postos    object
bairro            object
endereço          object
lat               object
lon               object
dtype: object

### 1.1.5. Verificar dados duplicados

In [23]:
df1_escolas.duplicated().sum()

13

### 1.1.6. Tratamento Inicial

In [24]:
df1_escolas = df1_escolas.drop_duplicates()

In [25]:
df1_escolas['lat'] = df1_escolas['lat'].str.replace(',', '.')
df1_escolas['lat'] = pd.to_numeric(df1_escolas['lat']).round(5)

df1_escolas['lon'] = df1_escolas['lon'].str.replace(',', '.')
df1_escolas['lon'] = pd.to_numeric(df1_escolas['lon']).round(5)

df1_escolas.dtypes

id                  int64
escolas_postos     object
bairro             object
endereço           object
lat               float64
lon               float64
dtype: object

In [26]:
df1_escolas = remove_acentos(df1_escolas)

In [27]:
df1_escolas = remodelando_id(df1_escolas)

In [28]:
df1_escolas = string_maisculo(df1_escolas)

In [29]:
tamanho_dos_dados(df1_escolas)

Colunas: 6
Linhas: 139


### 1.1.6. Salvar dados

In [30]:
salvar_dados_csv(df1_escolas, 'df1_escolas_descricao_dados.csv')

## 1.2. Material Didatico

### 1.2.1. Visão Geral dos Dados

In [31]:
df1_material_didatico = df_material_didatico_raw.copy()

In [32]:
df1_material_didatico.head().T

,0,1,2,3,4
id,178,634,483,476,132
Quantidade,20,121,220,190,67


In [33]:
df1_material_didatico.columns = renomeia_colunas(df1_material_didatico)

### 1.2.2. Tamanho dos dados

In [34]:
tamanho_dos_dados(df1_material_didatico)

Colunas: 2
Linhas: 151


In [35]:
qtd_total_unicos_por_coluna(df1_material_didatico)

Quantidade de valores unicos em id: 151
Quantidade de valores unicos em quantidade: 135


### 1.2.3. Verificar Nulos

In [36]:
df1_material_didatico.isna().sum()

id            0
quantidade    4
dtype: int64

### 1.2.4. Verificar tipos dos dados

In [37]:
df1_material_didatico.dtypes

id             int64
quantidade    object
dtype: object

### 1.2.5. Verificar dados duplicados

In [38]:
df1_material_didatico.duplicated().sum()

0

### 1.2.6. Tratamento Inicial

In [39]:
df1_material_didatico['quantidade'].unique()

array(['20', '121', '220', '190', '67', '397', '92', '578', '591', '473',
       '456', '223', '420', '573', "''", '366', '516', '319', '308',
       '111', '48', '525', '445', '164', '293', '152', '399', '185',
       '576', '63', '491', "'NaN'", '336', '159', '342', '262', '440',
       '365', nan, '183', '297', '44', '109', '350', '255', '457', '493',
       '489', '64', '530', '158', '123', '468', '234', '24', '218', '151',
       '564', '504', '61', '426', '167', '509', '292', '210', '153',
       "'397'", '41', '561', '180', '257', '260', '317', '345', '32',
       '513', '338', '416', '352', '393', '425', '136', "'395'", '115',
       '534', '430', '466', '51', '26', '94', '128', '520', '524', '279',
       '161', '179', '194', '599', '315', '435', '400', '294', '148',
       '269', '571', '235', '187', '392', '362', '71', '221', '372',
       '479', '99', '157', '296', '330', '316', '106', '207', '459',
       '537', '162', '579', '517', "' '", '135', '118', '341', '329',
     

In [40]:
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].str.replace("'", "")
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].replace(" ", np.nan)
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].replace('NaN', np.nan)

# Converter a coluna 'quantidade' para tipo numérico
df1_material_didatico['quantidade'] = pd.to_numeric(df1_material_didatico['quantidade'])

moda = df1_material_didatico['quantidade'].value_counts().idxmax()
df1_material_didatico['quantidade'] = df1_material_didatico['quantidade'].fillna(moda)


# Verificar se as correções foram aplicadas corretamente
print(df1_material_didatico['quantidade'].unique())


[ 20. 121. 220. 190.  67. 397.  92. 578. 591. 473. 456. 223. 420. 573.
 342. 366. 516. 319. 308. 111.  48. 525. 445. 164. 293. 152. 399. 185.
 576.  63. 491. 336. 159. 262. 440. 365. 183. 297.  44. 109. 350. 255.
 457. 493. 489.  64. 530. 158. 123. 468. 234.  24. 218. 151. 564. 504.
  61. 426. 167. 509. 292. 210. 153.  41. 561. 180. 257. 260. 317. 345.
  32. 513. 338. 416. 352. 393. 425. 136. 395. 115. 534. 430. 466.  51.
  26.  94. 128. 520. 524. 279. 161. 179. 194. 599. 315. 435. 400. 294.
 148. 269. 571. 235. 187. 392. 362.  71. 221. 372. 479.  99. 157. 296.
 330. 316. 106. 207. 459. 537. 162. 579. 517. 135. 118. 341. 329. 570.
 320. 335. 521. 225.]


In [41]:
df1_material_didatico = remodelando_id(df1_material_didatico)
df1_material_didatico['id'].unique()

array(['178', '634', '483', '476', '132', '410', '055', '490', '600',
       '089', '387', '331', '450', '558', '413', '698', '540', '309',
       '268', '523', '017', '035', '011', '226', '638', '271', '126',
       '180', '163', '361', '280', '060', '587', '653', '660', '382',
       '217', '378', '429', '025', '367', '373', '624', '157', '020',
       '117', '502', '351', '135', '647', '440', '259', '422', '041',
       '537', '230', '683', '146', '566', '388', '498', '663', '121',
       '161', '263', '118', '140', '260', '236', '090', '274', '676',
       '390', '608', '589', '256', '091', '285', '248', '441', '623',
       '603', '673', '182', '520', '412', '031', '514', '023', '415',
       '016', '570', '528', '414', '690', '202', '550', '211', '526',
       '492', '466', '636', '527', '586', '080', '398', '175', '308',
       '512', '576', '495', '379', '641', '689', '171', '477', '206',
       '687', '568', '399', '255', '262', '633', '616', '320', '144',
       '534', '342',

### 1.2.7. Salvar dados

In [42]:
salvar_dados_csv(df1_material_didatico, 'df1_material_didatico_descricao_dados.csv')

## 1.3. Sub-Prefeituras

### 1.3.1. Visão Geral dos Dados

In [43]:
df1_subprefeituras = df_subprefeituras_raw.copy()

In [44]:
df1_subprefeituras.head().T

,0,1,2,3,4
nome,Santo Cristo,Gamboa,Saúde,Caju,Centro
subprefeitura,Centro,Centro,Centro,Centro,Centro


### 1.3.2. Tamanho dos dados

In [45]:
tamanho_dos_dados(df1_subprefeituras)

Colunas: 2
Linhas: 164


In [46]:
qtd_total_unicos_por_coluna(df1_subprefeituras)

Quantidade de valores unicos em nome: 164
Quantidade de valores unicos em subprefeitura: 9


### 1.3.3. Verificar Nulos

In [47]:
df1_subprefeituras.isna().sum()

nome             0
subprefeitura    0
dtype: int64

### 1.3.4. Verificar tipos dos dados

In [48]:
df1_subprefeituras.dtypes

nome             object
subprefeitura    object
dtype: object

### 1.3.5. Verifica dados duplicados

In [49]:
df1_subprefeituras.duplicated().sum()

0

### 1.3.5. Tratamento Inicial

In [50]:
df1_subprefeituras['subprefeitura'] = df1_subprefeituras['subprefeitura'].str.upper()
df1_subprefeituras['nome'] = df1_subprefeituras['nome'].str.upper()

In [51]:
df1_subprefeituras.head().T

,0,1,2,3,4
nome,SANTO CRISTO,GAMBOA,SAÚDE,CAJU,CENTRO
subprefeitura,CENTRO,CENTRO,CENTRO,CENTRO,CENTRO


In [52]:
df1_subprefeituras = remove_acentos(df1_subprefeituras)

### 1.3.6. Salvar dados

In [53]:
salvar_dados_csv(df1_subprefeituras, 'df1_subprefeitura_descricao_dados.csv')

# 2.0. Feature Engineering

In [54]:
df2_escolas = df1_escolas.copy()
df2_material_didatico = df1_material_didatico.copy()
df2_subprefeituras = df1_subprefeituras.copy()

In [55]:
df2_escolas

,id,escolas_postos,bairro,endereço,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/NO,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858
...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUA BELCHIOR DA FONSECA 643,-23.00142,-43.63876
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/NO",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.O,-23.00908,-43.53758


In [56]:
df2_material_didatico

,id,quantidade
0,178,20.0
1,634,121.0
2,483,220.0
3,476,190.0
4,132,67.0
...,...,...
146,474,320.0
147,301,335.0
148,215,342.0
149,606,521.0


In [57]:
df2_subprefeituras

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


## 2.1. Merge dos dados

In [58]:
df2_merged = pd.merge(df2_escolas, df2_subprefeituras, left_on='bairro', right_on='nome', how='left')

In [59]:
df2_merged.sample(5).T

,136,46,52,72,103
id,301,388,230,091,379
escolas_postos,EM JONATAS SERRANO,EM CARDEAL CAMARA,EM BERLIM,E.M. PIAUI,CIEP NELSON MANDELA
bairro,GUARATIBA,PARADA DE LUCAS,OLARIA,GUADALUPE,CAMPO GRANDE
endereço,"ESTRADA DO MATO ALTO, S/NO","R. ANAMA, 98",PCA. RAMOS FIGUEIRA 20,AVENIDA BRASIL 23.364,"PRACA DANIEL LAMES, S/NO"
lat,-22.95316,-22.81527,-22.84779,-22.84542,-22.89558
lon,-43.57741,-43.30357,-43.26144,-43.37852,-43.57051
nome,GUARATIBA,PARADA DE LUCAS,OLARIA,GUADALUPE,CAMPO GRANDE
subprefeitura,ZONA OESTE,ZONA NORTE,ZONA NORTE,ZONA NORTE,ZONA OESTE


In [60]:
df2_merged = pd.merge(df2_merged, df2_material_didatico, left_on='id', right_on='id', how='inner')

In [61]:
df2_merged.sample(5).T

,9,30,83,106,5
id,089,587,016,171,017
escolas_postos,EM MARIO CLAUDIO,EM ALMIRANTE TAMANDARE,CIEP JOAO BATISTA DOS SANTOS,ESCOLA MUNICIPAL PADRE LEONEL FRANCA,E.M PRESIDENTE JOSE LINHARES
bairro,RIO COMPRIDO,VIDIGAL,CIDADE DE DEUS,REALENGO,IPANEMA
endereço,"R. HADDOCK LOBO, 148",AVENIDA PRESIDENTE JOAO GOULART 296,"RUA EDGAR WERNECK, 1.565",AVENIDA CANAL S/NDEG,"R. BARAO DA TORRE, 90"
lat,-22.91723,-22.9937,-22.94711,-22.88346,-22.98333
lon,-43.21104,-43.23558,-43.35974,-43.42945,-43.19904
nome,RIO COMPRIDO,VIDIGAL,CIDADE DE DEUS,REALENGO,IPANEMA
subprefeitura,CENTRO,ZONA SUL,JACAREPAGUA,GRANDE BANGU,ZONA SUL
quantidade,473.0,336.0,430.0,235.0,48.0


## 2.2. Tratamento de Nulos

In [62]:
df2_merged.isna().sum()

id                0
escolas_postos    0
bairro            0
endereço          0
lat               0
lon               0
nome              9
subprefeitura     9
quantidade        0
dtype: int64

In [63]:
df2_merged[df2_merged['subprefeitura'].isna()].sort_values(by='bairro')

,id,escolas_postos,bairro,endereço,lat,lon,nome,subprefeitura,quantidade
113,262,CIEP HERIVELTO MARTINS,AUGUSTO VASCONCELOS,"PRACA SANTO ANASTACIO, S/N",-22.88649,-43.52880,NaN,NaN,221.0
89,550,EM 25 DE ABRIL,FREGUESIA JPA,RUA MAMORE 78,-22.93816,-43.34258,NaN,NaN,128.0
92,492,EM MENEZES CORTES,FREGUESIA JPA,PRACA JOSE ALVES DE AZEVEDO 43,-22.94266,-43.33897,NaN,NaN,342.0
29,060,ESCOLA MUNICIPAL BARAO HOMEM DE MELO,MARACANA/ TIJUCA,"R. ALM. JOAO CANDIDO BRASIL, 352",-22.91683,-43.23692,NaN,NaN,342.0
28,280,ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO,MARACANA/ VILA ISABEL,"R. OITO DE DEZEMBRO, 275",-22.90982,-43.23939,NaN,NaN,491.0
131,068,EM WALQUIR PEREIRA,NOVA SEPETIBA,ESTRADA DE SEPETIBA S/N BLOCO 1,-22.95601,-43.68675,NaN,NaN,135.0
78,412,EM MOZART LAGO,OSWALDO CRUZ,RUA JOSE CARVALHO SALGADO S/N.O,-22.87221,-43.35033,NaN,NaN,395.0
85,528,CIEP MARGARET MEE,RECREIO,RUA MINISTRO ALIOMAR BALEEIRO S/NO,-23.01981,-43.48172,NaN,NaN,51.0
86,414,EM CLAUDIO BESSERMAN VIANNA - BUSSUNDA,RIO DAS PEDRAS,ESTRADA DE JACAREPAGUA 3327,-22.97336,-43.32916,NaN,NaN,26.0


In [64]:
# id = 262 | AUGUSTO VASCONCELOS = NaN

# id = 550 | FREGUESIA JPA = FREGUESIA (JACAREPAGUA) | JACAREPAGUA
df2_merged.loc[df2_merged['id'] == '550', 'nome'] = 'FREGUESIA (JACAREPAGUA)'
df2_merged.loc[df2_merged['id'] == '550', 'subprefeitura'] = 'JACAREPAGUA'

# id = 492 | FREGUESIA JPA = FREGUESIA (JACAREPAGUA)
df2_merged.loc[df2_merged['id'] == '492', 'nome'] = 'FREGUESIA (JACAREPAGUA)'
df2_merged.loc[df2_merged['id'] == '492', 'subprefeitura']  = 'JACAREPAGUA'

# id = 060 | MARACANA/ TIJUCA = MARACANA
df2_merged.loc[df2_merged['id'] == '060', 'nome'] = 'MARACANA'
df2_merged.loc[df2_merged['id'] == '060', 'subprefeitura'] = 'GRANDE TIJUCA'

# id = 280 | MARACANA/ VILA ISABEL  MARACANA
df2_merged.loc[df2_merged['id'] == '280', 'nome'] = 'MARACANA'
df2_merged.loc[df2_merged['id'] == '280', 'subprefeitura'] = 'GRANDE TIJUCA'

# id = 068 | NOVA SEPETIBA = NaN

# id = 412 | OSWALDO CRUZ = OSVALDO CRUZ
df2_merged.loc[df2_merged['id'] == '412', 'nome'] = 'OSVALDO CRUZ'
df2_merged.loc[df2_merged['id'] == '412', 'subprefeitura']= 'ZONA NORTE'

# id = 528 | RECREIO = RECREIO DOS BANDEIRANTES
df2_merged.loc[df2_merged['id'] == '528', 'nome'] = 'RECREIO DOS BANDEIRANTES'
df2_merged.loc[df2_merged['id'] == '528', 'subprefeitura'] = 'BARRA DA TIJUCA'

# id = 414 | RIO DAS PEDRAS = NaN

In [65]:
df2_merged[df2_merged['subprefeitura'].isna()].sort_values(by='bairro')

,id,escolas_postos,bairro,endereço,lat,lon,nome,subprefeitura,quantidade
113,262,CIEP HERIVELTO MARTINS,AUGUSTO VASCONCELOS,"PRACA SANTO ANASTACIO, S/N",-22.88649,-43.52880,NaN,NaN,221.0
131,068,EM WALQUIR PEREIRA,NOVA SEPETIBA,ESTRADA DE SEPETIBA S/N BLOCO 1,-22.95601,-43.68675,NaN,NaN,135.0
86,414,EM CLAUDIO BESSERMAN VIANNA - BUSSUNDA,RIO DAS PEDRAS,ESTRADA DE JACAREPAGUA 3327,-22.97336,-43.32916,NaN,NaN,26.0


## 2.3. Salvar dados

In [66]:
df2_merged.sample(5).T

,66,24,6,30,90
id,274,126,055,587,211
escolas_postos,EM ABEILARD FEIJO,EM DUQUE DE CAXIAS,EM SANTA CATARINA,EM ALMIRANTE TAMANDARE,EM FRANCIS HIME
bairro,CACUIA,GRAJAU,SANTA TERESA,VIDIGAL,TAQUARA
endereço,ESTRADA DA CACUIA 1386,"R. MAL. JOFRE, 74",RUA EDUARDO SANTOS 38,AVENIDA PRESIDENTE JOAO GOULART 296,ESTRADA PAU DA FOME 196
lat,-22.80769,-22.91854,-22.91554,-22.9937,-22.91883
lon,-43.18156,-43.26509,-43.19159,-43.23558,-43.41625
nome,CACUIA,GRAJAU,SANTA TERESA,VIDIGAL,TAQUARA
subprefeitura,ILHAS,GRANDE TIJUCA,CENTRO,ZONA SUL,JACAREPAGUA
quantidade,561.0,399.0,92.0,336.0,520.0


In [67]:
salvar_dados_csv(df2_merged, 'df2_feature_engineering.csv')